In [1]:
import os
os.chdir('../..')
!ls

product_impacts


In [28]:
import pandas as pd
import numpy as np
import pickle
import csv
import time
import ast
import random
from tqdm.notebook import tqdm
from googletrans import Translator
from easynmt import EasyNMT
from langdetect import detect, LangDetectException

import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# enter path here
data_dir = '../../future_of_food/openfoodfacts/all/'

In [4]:
chunks = 100000
data = pd.read_csv(f'{data_dir}en.openfoodfacts.org.products.csv', sep='\t', chunksize=chunks, low_memory=False, on_bad_lines='skip')

df_list = []

for chunk in data:
    df_list.append(chunk)

In [10]:
cols = ['product_name', 'categories', 'categories_tags', 'categories_en', 
        'origins', 'origins_tags', 'origins_en',
        'labels', 'labels_tags', 'labels_en',
        'stores', 'countries', 'countries_tags', 'countries_en', 
        'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags',
        'energy_100g', 'fat_100g', 'saturated-fat_100g', 
        'carbohydrates_100g', 'fiber_100g', 'proteins_100g',
        'sugars_100g', 'salt_100g']

df = pd.concat([df[cols] for df in df_list], axis=0, ignore_index=True)

In [16]:
df = df.fillna('')
df = df[df['product_name']!='']
df = df[df['product_name']!=' ']
df = df[df['countries_en']!='']

In [30]:
df['countries_en'] = df.apply(lambda row: row['countries_en'].split(','), axis=1)

In [33]:
df = df.explode('countries_en')

In [47]:
countries = df.groupby('countries_en')[['product_name']].count().reset_index()
df = df[df['countries_en'].isin(countries[countries['product_name']>1000]['countries_en'].values)]

In [50]:
df['product_name_lower'] = df['product_name'].str.lower()
df['ingredients_text_lower'] = df['ingredients_text'].str.lower()

df = df.drop_duplicates(subset=['product_name_lower', 'ingredients_text', 'countries_en'])
df = df.drop(['product_name_lower', 'ingredients_text_lower'], axis=1)

In [51]:
df.shape

(2583152, 25)

In [122]:
def detect_language(text):
    try:
        # Only attempt detection if text has sufficient length
        if len(text.strip()) > 1:  # Adjust this threshold as needed
            return detect(text)
        else:
            return "und"  # Assign "und" (undetermined) for empty or short text
    except LangDetectException:
        return "und"  # Assign "und" for undetectable languages

In [123]:
df['ingredients_text_language'] = df.progress_apply(lambda row: detect_language(row['ingredients_text'], axis=1)
df['product_name_language'] = df.progress_apply(lambda row: detect_language(row['product_name'], axis=1)

In [87]:
df['product_id'] = df.index
df['product_id'] = df['product_id'].apply(str)
df['product_id'] = '29112024' + df['product_id']
df['product_id'] = pd.to_numeric(df['product_id'])
first_column = df.pop('product_id') 
df.insert(0, 'product_id', first_column) 

In [ ]:
df['product_name_en'] = df['product_name']
df['ingredients_text_en'] = df['ingredients_text']

In [166]:
df_eng = df[(df['ingredients_text_language']=='en') 
            | ((df['ingredients_text_language']=='und') 
               & (df['product_name_language']=='en'))].reset_index(drop=True)

df_non_eng = df[(df['ingredients_text_language']!='en') 
                & (df['product_name_language']!='en')].reset_index(drop=True)

df_non_eng1 = df[(df['ingredients_text_language']!='en') 
                & (df['product_name_language']!='en')
                & (df['ingredients_text']!='')].reset_index(drop=True)

df_non_eng2 = df[(df['ingredients_text_language']!='en') 
                & (df['product_name_language']!='en')
                & (df['ingredients_text']=='')].reset_index(drop=True)

In [178]:
df_non_eng.shape

(1742836, 28)

In [168]:
df_non_eng1.shape

(556635, 28)

In [169]:
df_non_eng2.shape

(1186201, 28)

In [171]:
df_non_eng1[(df_non_eng1['product_name_language']==df_non_eng1['ingredients_text_language'])].shape

(300489, 28)

In [227]:
translation_cache = {}
translator = Translator()

def translate_with_cache(text):
    if text=='':
        return ''
    if text in translation_cache:
        return translation_cache[text]
    try:
        translated = translator.translate(text, dest='en').text
        translation_cache[text] = translated
        return translated
    except Exception as e:
        return f"Error: {e}"

In [ ]:
# Translate in batches with delays
batch_size = 100
for start in range(0, len(df_non_eng1), batch_size):
    end = start + batch_size
    print(f"Processing rows {start} to {end}")
    df_non_eng1.loc[start:end, 'ingredients_text_en'] = df_non_eng1['ingredients_text'][start:end].apply(translate_with_cache)
    print(df_non_eng1.loc[start:end,:].sample(1)[['ingredients_text', 'ingredients_text_en']].values)
    time.sleep(5)  # Add a delay to avoid rate limiting

Processing rows 0 to 100
[['mûre bio 50% (fruit 27%, purée 23%) sucre de canne bio, gélifiant: pectine de fruits, traces éventuelle de fruit à coques'
  'Organic blackberry 50%(fruit 27%, puree 23%) organic cane sugar, gelling: fruit pectin, possible traces of fruit in shells']]
Processing rows 100 to 200
[["Pâte (farine de blé tendre, eau, huile d'olive extra vierge, sel, levure), Sauce blanche (eau, crème de lait en poudre (proteines de lait, lait entier en poudre, amidon de mais modifié, sel), fromage bleu 7,9% équivalent à 1,7% du produit fini (lait, sel, ferment lactiques, presure microbienne, Penicillium roqueforti), huile de tournesol, poivre noir. amidon de mais) Fromage Mozzarella 16,4% (lait, sel, présure microbienne, ferments lactiques), Fromage Provolone piquant 5,8% (lait, sel, présure animale), Fromage Pecorino Romano AOP 29% (lait de brebis, sel, présure animale, ferments lactiques), Fromage Reibekase 2,9% (lait, sel, ferments lactiques, présure microbienne), huile de to

In [ ]:
df_non_eng1.head()

In [ ]:
df_non_eng1.to_csv(f'{data_dir}openfoodfacts_non_eng1_temp.csv', index=False)

In [ ]:
####### DELETE the subsequent 2 cells

In [ ]:
df_eng['product_name_en'] = df_eng['product_name']
df_eng['ingredients_text_en'] = df_eng['ingredients_text']

In [ ]:
df_non_eng2['ingredients_text_en'] = df_non_eng2['ingredients_text']

In [ ]:
# Translate in batches with delays
batch_size = 100
for start in range(0, len(df_non_eng1), batch_size):
    end = start + batch_size
    print(f"Processing rows {start} to {end}")
    df_non_eng1.loc[start:end, 'product_name_en'] = df_non_eng1['product_name'][start:end].apply(translate_with_cache)
    print(df_non_eng1.loc[start:end,:].sample(1)[['product_name', 'product_name_en']].values)
    time.sleep(5)  # Add a delay to avoid rate limiting

In [ ]:
# Translate in batches with delays
batch_size = 100
for start in range(0, len(df_non_eng2), batch_size):
    end = start + batch_size
    print(f"Processing rows {start} to {end}")
    df_non_eng2.loc[start:end, 'product_name_en'] = df_non_eng2['product_name'][start:end].apply(translate_with_cache)
    print(df_non_eng2.loc[start:end,:].sample(1)[['product_name', 'product_name_en']].values)
    time.sleep(5)  # Add a delay to avoid rate limiting

In [ ]:
#### BEFORE THIS, FILTER OUT TRANSLATIONS WITH ERRORS DECIDE WHETHER TO RUN TRANSLATION ON THEM AGAIN

In [ ]:
df = pd.concat([df_eng, df_non_eng1, df_non_eng2], axis=0, ignore_index=True)

In [ ]:
df.to_csv(f'{data_dir}openfoodfacts_lang.csv', index=False)